In [ ]:
READ ME

##### 순서  ######

1. host ip를 원격 컴퓨터 ip로 바꿔주세요.

2. 원격 컴퓨터에서 주피터를 설치하기 전에 무조건
이 고 있는 로컬 컴퓨터에서 ~/.ssh/config에 host의 ssh ip가 존재해야합니다.
따라서 에러가 발생하는 경우에
47번째 줄 #local_ssh_config() 
위의 주석을 해제하시고 함수를 실행시켜주세요.

3. install_in() 함수를 실행하세요.

4. 주피터가 자동으로 설치됩니다!


In [5]:
import subprocess
import os
# 로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행하세요
# 그 후에 다시 주석 해제해주세요.
user = "root"
host = "172.16.254.12"
password = "asd123!@"


########################### 로컬 ###########################
## 우선 로컬에서 원격 호스트 IP에 대한 StrictHostKeyChecki0 ng=no를 항상 허용해줍니다.

def local_ssh_config():
    ssh_dir = os.path.expanduser("~/.ssh")
    config_path = os.path.join(ssh_dir,"config")
    
    os.makedirs(ssh_dir, exist_ok=True)
    os.chmod(ssh_dir, 0o700)
    
    config_content = f"""
    Host {host}
        StrictHostKeyChecking no
        UserKnownHostsFile=/dev/null
    """
    
    # 이미 같은 설정이 있는지 확인하고 없으면 추가
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            content = f.read()
    else:
        content = ""
    
    if host not in content:
        with open(config_path, "a") as f:
            f.write(config_content)
        print(f"SSH 설정 추가 완료: {host}")
    else:
        print(f"이미 설정되어 있습니다: {host}")
    
    # 권한 설정 (보안상 필요)
    os.chmod(config_path, 0o600)

def local_set():
    print("로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행")
    os.system("dnf install -y expect")

#local_ssh_config()
#local_set()

########################### 원격 ###########################

def cc(command):
    cmd = f"sshpass -p '{password}' ssh {user}@{host} '{command}'"
    result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
    print(result.stdout)

def install_packages():
    # 여러 명령을 한 줄로 연결하기 (&& 로 순차 실행)
    commands = """
     dnf -y install net-tools &&
     dnf -y install vim &&
     dnf install -y epel-release &&
     dnf -y update && dnf -y upgrade
     dnf -y install wget
     dnf -y install expect
    """
    cc(commands)

def install_miniconda_and_config():
    print("Miniconda를 설치합니다.")
    print("/etc/profile.d/conda.sh 파일에 Miniconda를 등록해서 , 터미널 어디서든 conda 명령을 쓸 수 있게 만드는 설정입니다.")
    commands = """
cd ~
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
bash miniconda.sh -b -p /opt/miniconda3
echo 'export PATH="/opt/miniconda3/bin:$PATH"' >> /etc/profile.d/conda.sh
source /etc/profile.d/conda.sh
"""
    cc(commands)

def install_conda_and_paramiko():
    print("conda를 설치합니다")
    commands = """
/opt/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main 
/opt/miniconda3/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r 
/opt/miniconda3/bin/conda install -y jupyter
"""
    cc(commands)
    cc("echo 'source /etc/profile.d/conda.sh' >> /etc/bashrc")
    cc("python -m pip install paramiko")

    

def set_firewalld_and_selinux_off():
    print("firewalld , selinux 방화벽을 끕니다")
    cc("systemctl stop firewalld")
    commands = f"""
sudo sed -i 's/^SELINUX=.*/SELINUX=disabled/' /etc/selinux/config
"""
    cc(commands)

def set_jupyter_passwd():
    
    passwd = input("주피터 패스워드 비밀번호를 설정하세요 (ex. asd123!@)")
    #cc("sudo bash -c 'source /etc/profile.d/conda.sh && /usr/bin/expect /root/jupyter_passwd.exp'")
    commands = f"""
cat << EOF > /root/jupyter_passwd.exp
#!/usr/bin/expect -f
spawn /opt/miniconda3/bin/jupyter server password
expect "Enter password"
send "{passwd}\\r"
expect "Verify password"
send "{passwd}\\r"
expect eof
EOF

chmod +x /root/jupyter_passwd.exp

sudo bash -c "source /etc/profile.d/conda.sh && /usr/bin/expect /root/jupyter_passwd.exp"
"""


    cc(commands)


# 재부팅 시 자동 활성화 설정입니다.
def set_systemd_service():
    commands = f"""
cat << EOF > /etc/systemd/system/jupyter.service
[Unit]
Description=Jupyter Notebook
After=network.target

[Service]
Type=simple
User=root
# 설치 경로를 /opt로 변경했으므로 WorkingDirectory도 /opt로 설정하는 것이 좋음
WorkingDirectory=/opt
# ExecStart의 모든 경로를 /opt/miniconda3 기준으로 수정
ExecStart=/opt/miniconda3/bin/jupyter notebook --config=/root/.jupyter/jupyter_server_config.json --ip={host} --port=8000 --no-browser --allow-root

[Install]
WantedBy=multi-user.target
EOF

sudo systemctl daemon-reload
sudo systemctl enable jupyter
sudo systemctl start jupyter
sudo systemctl status jupyter
"""
    cc(commands)

# 실행
def install_in():
 #   install_packages()
 #   install_miniconda_and_config()   
    install_conda_and_paramiko()
    set_firewalld_and_selinux_off()
    set_jupyter_passwd()
    set_systemd_service()
    
install_in()


conda를 설치합니다
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.





CalledProcessError: Command 'sshpass -p 'asd123!@' ssh root@172.16.254.12 'python -m pip install paramiko'' returned non-zero exit status 127.